In [ ]:
%matplotlib inline

import datajoint as dj
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np

import seaborn as sns
sns.set_style('ticks', rc={'image.cmap': 'bwr'})

import os
import sys
import inspect

p = !pwd
p = os.path.dirname(os.path.dirname(p[0]))
if p not in sys.path:
    sys.path.append(p)

In [ ]:
from cnn_sys_ident.mesonet.data import MultiDataset
from cnn_sys_ident.mesonet.vis import MEIParams, MEIGroup, MEI
from cnn_sys_ident.mesonet.parameters import Core, Readout, Model, RegPath, Fit
from cnn_sys_ident.architectures.training import Trainer
from cnn_sys_ident.mesonet import MODELS

In [ ]:
data_key = {'data_hash': 'cfcd208495d565ef66e7dff9f98764da'}
dataset = MultiDataset() & data_key

In [ ]:
model_rel = MODELS['HermiteSparse'] * dataset \
    & 'positive_feature_weights=False AND shared_biases=False' \
    & 'num_filters_2=16'
key = (Fit() * model_rel).fetch(dj.key, order_by='val_loss', limit=1)[0]
model = Fit().load_model(key)

In [ ]:
w = model.base.evaluate(model.core.weights_all[0])
print(w.shape)

In [ ]:
fig, axes = plt.subplots(8, 16, figsize=(20, 10))
for wi, ax in zip(w[:,:,0,:].transpose([2, 0, 1]), axes.flatten()):
    m = np.abs(wi).max()
    ax.imshow(wi, vmin=-m, vmax=m)
    ax.axis('off')

fig.savefig('figures/kernels.eps', format='eps')

# Visualize Hermite functions

In [ ]:
from cnn_sys_ident.utils.hermite import hermite_2d, rotation_matrix

In [ ]:
filter_size = 13
upsampling = 2
k = filter_size
H, desc, mu = hermite_2d(k, filter_size*upsampling, xvalmax=2*np.sqrt(k))
N = H.shape[0]
m = int(np.ceil(np.sqrt(N)))
n = int(np.ceil(N / m))
mx = H.max()
fig, axes = plt.subplots(m, n, figsize=(n, m))
for ax, h in zip(axes.flatten(), H):
    ax.imshow(h, vmin=-mx, vmax=mx)
    ax.axis('off')

fig.savefig('figures/hermite_fn.eps', format='eps')